In [3]:
# Se importa librería encargada de cargar los procesos para imágenes
import cv2
# Se importa numpy para el manejo de array
import numpy as np 
# Se importa os para la lectura de directorio de imágenes
from os import listdir
# Se importa pyplot para mostrar imágenes y gráficos
import matplotlib.pyplot as plt
# Se importa pandas para trabajar con csv
import pandas as pd
# Se importa math para funciones
import math as mt
# Se importa joblib para poder cargar y guardar modelos
import pickle


# Se importa skimage feature para la extración de características de GLCM
from skimage.feature import greycomatrix, greycoprops

# Se importan medidas de desempeño y error
from sklearn import metrics
# Se importan Repeated K-Fold Cross Validator para generar los conjuntos de Entrenamiento y Test 
from sklearn.model_selection import RepeatedKFold
# Se importa la función para el calculo de la precisión
from sklearn.metrics import accuracy_score
# Se importa la función para obtener la matriz de confusión
from sklearn.metrics import confusion_matrix
# Se importa el divisor de data
from sklearn.model_selection import train_test_split

# Se importa el modelo de clasificación correspondiente a una SVM en este caso SVC (C-support vector classification)
from sklearn.svm import SVC

# Se importa StandardScaler para poder estandarizar los valores de los features para tener la misma escala 
from sklearn.preprocessing import StandardScaler
# Se importa ssim para poder calcular la diferencia entre imágenes
from skimage.measure import compare_ssim as ssim

# Se importa para poder calcular medidas de tendencia centrales
from scipy import stats

In [4]:
#np.random.seed(100)

class BatAlgorithm():
    def __init__(self, D, NB, N_Gen, A0, r0, alpha, gamma, Fmin, Fmax, LowerB, UpperB, function, imgSamples, dataSamples):
        
        # Inicialización de Parámetros
        self.D = D  #dimension
        self.NB = NB  #number of bats 
        self.N_Gen = N_Gen  #generations
        self.alpha = alpha  # loudness update
        self.gamma = gamma  # pulse rate update
        self.r0 = r0 # Pulse rate initial
        self.A0 = A0 # Loudnes initial
        self.Fmin = Fmin  #frequency min
        self.Fmax = Fmax  #frequency max
        self.LowerB = LowerB  #lower bound
        self.UpperB = UpperB  #upper bound
        self.Funct = function #function to optimize
        
        # Data Training
        self.imgSamples = imgSamples
        self.dataSamples = dataSamples
        
        # Inicializar Loudness (A) y Pulse rate (R)
        self.A = [self.A0 for i in range(self.NB)]
        self.r = [self.r0 for i in range(self.NB)]
        
        # Inicializar limite inferior y superior para cada bat
        self.Lb = [[0.0 for i in range(self.D)] for j in range(self.NB)]  #lower bound
        self.Ub = [[0.0 for i in range(self.D)] for j in range(self.NB)]  #upper bound
        
        # Inicializar frecuencia
        self.F = [0.0] * self.NB  #frequency

        # Inicializar velocidad de los bats
        self.v = [[0.0 for i in range(self.D)] for j in range(self.NB)]  #velocity
        
        # Inicilizar soluciones de los bats
        self.Sol = [[0.0 for i in range(self.D)] for j in range(self.NB)]  #population of solutions
        
        # Inicilizar fitness de cada bat y el mejor fitness
        self.Fitness = [0.0] * self.NB  #fitness
        self.f_max = 0.0  #maximum fitness
        
        # Se agregan las variables de matriz de confusión
        self.confM = [0.0] * self.NB #matriz de confusión
        self.maxconfM = 0 #máxima matriz de confusión
        
        # Inicilizar la mejor solución encontrada
        self.best = [0.0] * self.D  #best solution
        
        
    def best_bat(self):
        i = 0
        j = 0
        # Se obtiene el mejor bat de toda la 1ra generación
        for i in range(self.NB):
            if self.Fitness[i] > self.Fitness[j]: # Cambiar para max o min
                j = i
        # Se actualiza el fitness mejor y la sol para la 1ra vez
        for i in range(self.D):
            self.best[i] = self.Sol[j][i]
        self.f_max = self.Fitness[j]
        self.maxconfM = self.confM[j]

    def init_bat(self):
        # Iniciliza los limites para cada bat y variables
        for i in range(self.NB):
            for j in range(self.D):
                self.Lb[i][j] = self.LowerB[j]
                self.Ub[i][j] = self.UpperB[j]
                
        # Se inicializan las soluciones para cada bat
        for i in range(self.NB):
            self.F[i] = 0
            for j in range(self.D):
                rnd = np.random.uniform(0, 1)
                self.v[i][j] = 0.0
                self.Sol[i][j] = self.Lb[i][j] + (self.Ub[i][j] - self.Lb[i][j]) * rnd
            self.Fitness[i], self.confM[i] = self.Funct(self.D, self.Sol[i], self.imgSamples, self.dataSamples)
        
        # Se selecciona el mejor bat de la generacion?
        self.best_bat()

    def simplebounds(self, val,index):
        # Función encargada de normalizar las soluciones en los limites
        if(index == 0):
            if val > self.UpperB[index]:
                val = self.UpperB[index]
            if val < self.LowerB[index]:
                val = self.LowerB[index]
        if(index == 1):
            if val > self.UpperB[index]:
                val = self.UpperB[index]
            if val < self.LowerB[index]:
                val = self.LowerB[index]
    
        return val

    def move_bat(self):
        # Se inicializa matriz de soluciones 
        S = [[0.0 for i in range(self.D)] for j in range(self.NB)]

        self.init_bat()

        for t in range(self.N_Gen):
            factor = np.mean(self.A)
            for i in range(self.NB):
                rnd = np.random.uniform(0, 1)
                self.F[i] = self.Fmin + (self.Fmax - self.Fmin) * rnd
                
                for j in range(self.D):
                    self.v[i][j] = self.v[i][j] + (self.best[j] - self.Sol[i][j]) * self.F[i]
                    S[i][j] = self.Sol[i][j] + self.v[i][j]

                    S[i][j] = self.simplebounds(S[i][j],j)

                rnd = np.random.uniform(0,1)

                if rnd > self.r[i]:
                    for j in range(self.D):
                        rnd = np.random.uniform(-1.0,1.0)
                        S[i][j] = self.best[j] + rnd*factor
                        S[i][j] = self.simplebounds(S[i][j],j)
                        
                Fnew, Mconf = self.Funct(self.D, S[i], self.imgSamples, self.dataSamples)

                rnd = np.random.uniform(0,1)

                if (Fnew > self.Fitness[i]) and (rnd < self.A[i]): # Cambiar para max o min
                    for j in range(self.D):
                        self.Sol[i][j] = S[i][j]
                    self.Fitness[i] = Fnew
                    self.confM[i] = Mconf

                if Fnew > self.f_max: # Cambiar para max o min
                    for j in range(self.D):
                        self.best[j] = self.Sol[i][j]
                    self.f_max = Fnew
                    self.maxconfM = Mconf
                    
                    self.A[i] = self.A[i]*self.alpha
                    self.r[i] = self.r0*(1 - mt.exp(-1*self.gamma*i))
            print("\nFitness de generación:",self.Fitness)
            print("Fitness mejor encontrado:",self.f_max)
            #print("Matriz de Confusión:\n",self.confM)
            print("Solucion:",self.best)
                    
        print ("\nFitness máximo: ",self.f_max)
        print ("Solución para máximo: ",self.best)
        print ("Matriz de Confusión: ",self.maxconfM)
        return self.best, self.maxconfM

In [5]:
# Función encargada de cargar las imágenes y obtener sus features
def cargarFotosYFeatures(path):
    casos = listdir(path)
    featureList = []
    contadorCaso = 0
    
    #------------- SECCIÓN ENCARGADA DE CREAR LAS IMÁGENES A COMPARAR EN LA SSIM -------------#
    imgNueva = np.zeros((75,100))
    imgNueva2 = np.zeros((75,100))
    lineThickness = 1
    cv2.line(imgNueva, (0, 75), (100, 0), (255,255,255), lineThickness)
    cv2.line(imgNueva2, (0, 75), (100, 45), (255,255,255), lineThickness)
    #------------- SECCIÓN ENCARGADA DE CREAR LAS IMÁGENES A COMPARAR EN LA SSIM -------------#
    
    # Se cargan las imágenes y se extraen sus features
    contador = 1
    for caso in casos:
        nombreImgs = listdir(path+caso+"/")
        featureCaso = []
        for nombre in nombreImgs:
            if(nombre != "Thumbs.db"):
                img = cv2.imread(path+caso+"/"+nombre)
                img = getROIyFeatures(img,imgNueva,imgNueva2)
                featureCaso.append(img)
                contador = contador + 1
        featureList.append(featureCaso)
        contadorCaso = contadorCaso +1
        if(contadorCaso%50 == 0):
            print("Cargadas fotografías hasta caso: "+caso)
    print("Fotografías cargadas: "+str(contador))
    return featureList

In [6]:
# Función encargada de obtener ROI y features de estos
def getROIyFeatures(img,imgNueva,imgNueva2):
    # Se obtiene las 3 ROI a analizar
    imgROI1 = img[0:36,50:100] 
    imgROI2 = img[36:75,0:50] 
    imgROI3 = img[36:75,50:100] 
    
    # Se obtienen las features de cada ROI
    features1 = np.array(getFeatures(imgROI1))
    features2 = np.array(getFeatures(imgROI2))
    features3 = np.array(getFeatures(imgROI3))
    
    # Se obtienen las features de la imagen para SSIM y MSE
    features4 = np.array(getSSIMyMSE(img,imgNueva,imgNueva2))

    # Se concatenan todas en un arreglo de 32 features
    features = np.concatenate((features1,features2,features3,features4),axis= None)
    
    return features

In [7]:
# Función encargada de obtener los features por imagen
def getFeatures(img):
    features = []
    
    #------------------ HISTOGRAM ----------------------#
    # Features de histograma
    hist = histogram(img)
    
    mean = np.mean(hist)
    std = np.std(hist) 
    
    features.append(mean)
    features.append(std)
    
    #------------------ HISTOGRAM ----------------------#
    
    img = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
        
    #------------------ GLCM ---------------------------#
    
    # Features de GLCM
    glcm = greycomatrix(img, [1], [0], 256, symmetric=True, normed=True)
    dissimilarity = greycoprops(glcm, 'dissimilarity')
    contrast = greycoprops(glcm, 'contrast')
    homogeneity = greycoprops(glcm, 'homogeneity')
    ASM = greycoprops(glcm, 'ASM')
    energy = greycoprops(glcm, 'energy')
    correlation = greycoprops(glcm, 'correlation')
    
    # Características en 0 grados
    features.append(float(dissimilarity))
    features.append(float(contrast))
    features.append(float(homogeneity))
    features.append(float(ASM))
    features.append(float(energy))
    features.append(float(correlation))
    
    #------------------ GLCM ---------------------------#
    
    return features

In [8]:
# Función encargada de calcular el histograma de colores de una imagen
def histogram(img):
    color = ('b','g','r')
    for i, c in enumerate(color):
        hist = cv2.calcHist([img], [i], None, [256], [0, 256])
    return hist

In [9]:
# Función encargada de calcular el error cuadrático medio entre 2 imágenes
def mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

In [10]:
# Función encargada de calcular el índice de similaridad estructural
def getSSIMyMSE(img,imgNueva,imgNueva2):
    img = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(img,100,200,apertureSize = 5)
    
    # Se aplica una máscara para obtener la 1ra sección a comparar por SSIM y MSE
    mask = np.zeros(edges.shape, dtype=np.uint8)
    roi_corners = np.array([[(90,10),(0,75),(90,0)]], dtype=np.int32)
    ignore_mask_color = (255,)
    cv2.fillPoly(mask, roi_corners, ignore_mask_color)
    masked_image = cv2.bitwise_and(edges, mask)
    
    # Se aplica una máscara para obtener la 2da sección a comparar por SSIM y MSE
    mask2 = np.zeros(edges.shape, dtype=np.uint8)
    roi_corners2 = np.array([[(90,60),(0,75),(90,40)]], dtype=np.int32)
    ignore_mask_color2 = (255,)
    cv2.fillPoly(mask2, roi_corners2, ignore_mask_color2)
    masked_image2 = cv2.bitwise_and(edges, mask2)
    
    # Se compara cada sección con la correspondiente recta para obtener las features
    ssim_const = ssim(masked_image,imgNueva,data_range=imgNueva.max() - imgNueva.min())
    MSE = mse(masked_image,imgNueva)
    
    ssim_const2 = ssim(masked_image,imgNueva2,data_range=imgNueva.max() - imgNueva.min())
    MSE2 = mse(masked_image,imgNueva2) 
    
    ssim_const3 = ssim(masked_image2,imgNueva,data_range=imgNueva.max() - imgNueva.min())
    MSE3 = mse(masked_image2,imgNueva)
    
    ssim_const4 = ssim(masked_image2,imgNueva2,data_range=imgNueva.max() - imgNueva.min())
    MSE4 = mse(masked_image2,imgNueva2)
    
    # Se agrega cada feature a un arreglo a retornar
    features = []
    features.append(ssim_const)
    features.append(ssim_const2) 
    features.append(ssim_const3) 
    features.append(ssim_const4)
    features.append(MSE)
    features.append(MSE2) 
    features.append(MSE3) 
    features.append(MSE4)
    
    return features

In [11]:
# Se la información de los labels de cada imagen
dataY = pd.read_csv("images_250/GRIGRI_Training_250.csv",header=None)
dataY = pd.Series.ravel(dataY)

In [12]:
# Se cargan las fotografías y se obtienen sus features
featuresImgs = cargarFotosYFeatures("images_250/dirTraining/")

Cargadas fotografías hasta caso: 071
Cargadas fotografías hasta caso: 125
Cargadas fotografías hasta caso: 182
Cargadas fotografías hasta caso: 241
Cargadas fotografías hasta caso: 301
Fotografías cargadas: 5001


In [13]:
# Se preparan los datos y labels para utilizarlos en la SVM
imgSamples = []
dataSamples = []
for featureArray,label in zip(featuresImgs,dataY):
    for featuresImg in featureArray:
        imgSamples.append(featuresImg)
        dataSamples.append(label)
imgSamples = np.array(imgSamples)
dataSamples = np.array(dataSamples)

In [14]:
print(imgSamples[0])
print(dataSamples)

[7.03125000e+00 9.94633026e+01 1.67517007e+01 3.03686508e+03
 8.52264619e-01 7.19409447e-01 8.48180080e-01 5.96968212e-01
 7.61718750e+00 1.00648178e+02 3.05306122e+01 5.48792360e+03
 7.54851656e-01 5.61961362e-01 7.49640822e-01 4.69728669e-01
 7.61718750e+00 9.75972748e+01 2.74296180e+01 4.85718786e+03
 7.42062090e-01 5.35533354e-01 7.31801444e-01 5.98688534e-01
 8.28640987e-01 7.33960115e-01 6.51166252e-01 7.94100696e-01
 1.59528000e+03 1.89006000e+03 3.26859000e+03 2.78307000e+03]
[1 1 1 ... 1 1 1]


In [15]:
print(imgSamples[0])
print(dataSamples)

[7.03125000e+00 9.94633026e+01 1.67517007e+01 3.03686508e+03
 8.52264619e-01 7.19409447e-01 8.48180080e-01 5.96968212e-01
 7.61718750e+00 1.00648178e+02 3.05306122e+01 5.48792360e+03
 7.54851656e-01 5.61961362e-01 7.49640822e-01 4.69728669e-01
 7.61718750e+00 9.75972748e+01 2.74296180e+01 4.85718786e+03
 7.42062090e-01 5.35533354e-01 7.31801444e-01 5.98688534e-01
 8.28640987e-01 7.33960115e-01 6.51166252e-01 7.94100696e-01
 1.59528000e+03 1.89006000e+03 3.26859000e+03 2.78307000e+03]
[1 1 1 ... 1 1 1]


In [16]:
# Se revisa la estructura de los datos previo al entrenamiento
print(imgSamples.shape)
print(dataSamples.shape)

(5000, 32)
(5000,)


In [17]:
# Se separan los datos en 80% y 20%
imgSamplesTrainF, imgSamplesTest, dataSamplesTrainF, dataSamplesTest = train_test_split(imgSamples, dataSamples, 
                                                                                      test_size=0.20, random_state=0)
# Se separan los datos de training en training y validation 90% y 10% 
imgSamplesTrain, validationSet, dataSamplesTrain, dataValidationSet = train_test_split(imgSamplesTrainF, dataSamplesTrainF, 
                                                                                      test_size=0.20, random_state=0)

print("Training:")
print(imgSamplesTrain.shape)
print(dataSamplesTrain.shape)
print("Validación:")
print(validationSet.shape)
print(dataValidationSet.shape)
print("Testing:")
print(imgSamplesTest.shape)
print(dataSamplesTest.shape)

Training:
(3200, 32)
(3200,)
Validación:
(800, 32)
(800,)
Testing:
(1000, 32)
(1000,)


In [18]:
# Función encargada de entrenar un modelo de SVM (esta función es utilizada por el algoritmo de optimización en un Bat)
def optimizeModel(D,sol,imgSamples,dataSamples):
    # Se generan arreglos para guardar las métricas para el modelo a entrenar
    accuracyClasificacion = []
    confusionM = 0
    # Se generan los conjuntos para 5 Folds con 10 iteraciones del proceso y random_state fijado para obtener los mismos resultados
    RepKFoldVal = RepeatedKFold(n_splits=5, n_repeats=2, random_state=1)

    # Se realiza una iteración para generar los conjuntos de entrenamiento y test luego del Repeated K-Fold Cross Validation
    for indice_train, indice_test in RepKFoldVal.split(imgSamples,dataSamples):
        # Se obtiene los arreglos con los conjuntos de entrenamiento y test
        X_train, X_test = imgSamples[indice_train], imgSamples[indice_test]
        Y_train, Y_test = dataSamples[indice_train], dataSamples[indice_test]

        # Se escalan los datos
        sc = StandardScaler()
        sc.fit(X_train)
        X_train = sc.transform(X_train)
        X_test  = sc.transform(X_test)        
        
        # Se instancia un clasificador de tipo SVM
        svmClass = SVC(gamma=sol[0],kernel='rbf',C=sol[1])
        svmClass.fit(X_train,Y_train)

        # Se obtienen las predicciones del modelo SVM
        predictSVM = svmClass.predict(X_test)
        exactitud = svmClass.score(X_test,Y_test)
        
        # Se obtiene la matriz de confusión del modelo
        confusionM = confusionM + confusion_matrix(Y_test, predictSVM)
        
        # Se obtiene la precisión del modelo SVM para esta iteración
        accuracyClasificacion.append(exactitud)
        
    return np.mean(accuracyClasificacion), confusionM/10

In [ ]:
# Se instancian los parámetros del Bat y se realiza el proceso de optimización
ba = BatAlgorithm(2,10,20,0.95,0.1,0.95,0.95,0,1,[1/32, 4],[1, 8],optimizeModel,validationSet,dataValidationSet)
parametros, matriz  = ba.move_bat()

In [ ]:
# Se escalan los datos
sc = StandardScaler()
sc.fit(imgSamplesTrain)
imgSamplesTrainScaled = sc.transform(imgSamplesTrain)
imgSamplesTestScaled  = sc.transform(imgSamplesTest)

# Entrenar modelo definitivo
clfClass = SVC(gamma=parametros[0],kernel='rbf',C=parametros[1])
clfClass.fit(imgSamplesTrainScaled,dataSamplesTrain)

# Se obtiene la exactitud y la matriz de confusión
exactitud = clfClass.score(imgSamplesTestScaled,dataSamplesTest)
predictClf = clfClass.predict(imgSamplesTestScaled)
confusionM = confusion_matrix(dataSamplesTest, predictClf)

print("Exactitud: "+str(exactitud))
print("Matriz de Confusión: "+str(confusionM))